# Climatiq notebook examples
[Climatiq](https://climatiq.io/) provides an API to help you estimate how much greenhouse gasses your activities emit, based on validated conversion factors known as “emission factors.”  
An emission factor is a way to convert different human activities into greenhouse gases.
It will take that human activity and estimate how much CO2 equivalent (abbreviated CO2e, usually expressed in kilograms), that activity will emit.

Different emission factors can help us answer questions like:
- How much CO2e is emitted taking the train from Paris to Berlin?
- How much CO2e is emitted spending $1,000 on soft drinks in the US?
- How much CO2e is emitted staying a night at a hotel in China?

Climatiq has a wide variety of emission factors available, that you can browse in our [Data Explorer](https://explorer.climatiq.io/), to get a feel for the sort of questions Climatiq can help you answer.

# API Key
All calls to the Climatiq API require that you have an account and that you send your API key. Signing up is free and takes only a few minutes.

[Click here](https://docs.climatiq.io/guides/getting-api-key) to see how to sign up and get your API key, which you should input below.

In [32]:
MY_API_KEY = input() #INPUT YOUR CLIMATIQ API KEY HERE

In [33]:
# This block installs requests, which is a python library for doing http requests
# Install requests using pip in the current python kernel
# See https://jakevdp.github.io/blog/2017/12/05/installing-python-packages-from-jupyter/ for details
# Note that if you are using conda you might need to uncomment the later line
import sys
!{sys.executable} -m pip install requests
# Delete the above line, and uncomment the below line if you are using conda
# !conda install --yes --prefix {sys.prefix} numpy

import requests
import pprint

Defaulting to user installation because normal site-packages is not writeable


# Documentation
This python notebook will mostly focus on some quick examples so you can get a feel for the API.
If you'd like a little more introduction, check our out [quickstart guide](https://docs.climatiq.io/guides/quickstart)

The notebook is tested using python3.10, but should work in python3.6 and above.

We also won't cover everything the API can do in this notebook, but check out the [API documentation](https://docs.climatiq.io/) for a lot more comprehensive documentation.

# Searching
If you need to figure out which emission factors are available, the easiest way is to try out our [Data Explorer](https://explorer.climatiq.io/) that allows you to search for, and filter emission factors based on sources, regions or free-text searching.

We also have a `search` rendpoint available for searching. It will let you do free-text searching, and optionally filtering on sources, regions etc.
See how to use it below.

In [34]:
url = "https://beta3.api.climatiq.io/search"
query="grid mix"

query_params = {
    # Free text query can be writen under the query string
    "query": query,
    # You can also filter on region, year, source and more
    "region": "AT"
}

# You must always specify your AUTH token in the "Authorization" header like this.
authorization_headers = {"Authorization": f"Bearer: {MY_API_KEY}"}


response = requests.get(url, params=query_params, headers=authorization_headers).json()
# The search endpoint returns a lot of stuff - here are the keys
print(response.keys())

# The most relevant is probably the results - here are the first three results
response["results"][0:3]

dict_keys(['current_page', 'last_page', 'total_results', 'results', 'possible_filters'])


[{'id': 'electricity-energy_source_grid_mix',
  'name': 'Grid mix',
  'category': 'Electricity',
  'sector': 'Energy',
  'source': 'AIB',
  'source_link': 'https://www.aib-net.org/facts/european-residual-mix',
  'uncertainty': None,
  'year': '2020',
  'region': 'AT',
  'region_name': 'Austria',
  'description': 'Emission intensity of grid mix as reported. Published by the Association of Issuing Bodies (AIB). Retrieved from 2020 European Residual Mix Report. Includes production fuel mix factor.NOTE: this value is provided in CO2 only and does not include the impact of other GHGs.',
  'unit_type': ['Energy'],
  'unit': 'kg/kWh',
  'lca_activity': 'electricity_generation',
  'supported_calculation_methods': ['ar4'],
  'factor': 0.11118,
  'factor_calculation_method': 'ar4',
  'factor_calculation_origin': 'source',
  'constituent_gases': {'co2e_total': 0.11118,
   'co2e_other': None,
   'co2': None,
   'ch4': None,
   'n2o': None}},
 {'id': 'electricity-energy_source_grid_mix',
  'name': 

# List all emission factors
If you need to do some processing locally, you can also fetch a list of all emission factors to use locally through the `emission-factors` endpoint.

You can view the full API documentation for [this endpoint here](https://docs.climatiq.io/#list-emission-factors)

In [35]:
url = "https://beta3.api.climatiq.io/emission-factors"

headers = {
    # You must always specify your AUTH token in the "Authorization" header like this.
    "Authorization": f"Bearer: {MY_API_KEY}",
    # The list of emission factors is lare. Enable gzip compression to save some bandwidth here.
    "Accept-encoding": "gzip"
    }


response = requests.get(url, headers=headers)
response.json()


{'results': [{'id': 'accommodation_type_holiday',
   'name': 'Holiday (travel/accommodation)',
   'category': 'Accommodation',
   'sector': 'Restaurants and Accommodation',
   'source': 'BEIS',
   'source_link': 'https://www.gov.uk/government/statistics/uks-carbon-footprint',
   'uncertainty': None,
   'year': '2018',
   'region': 'GB',
   'region_name': 'United Kingdom',
   'description': 'Emission intensity of supply chain in GBP spend on: holiday in the uk/ holiday abroad/ room hire. In actual prices in £s and including VAT. Provided by BEIS/DEFRA in the official report on GHG Emissions of the UK between 1996-2018. Where source emission factors are identical across a variety of activities they have been grouped to avoid confusion - details are documented in the OEFDB data guidance.',
   'unit_type': ['Money'],
   'unit': 'kg/GBP',
   'lca_activity': 'unknown',
   'supported_calculation_methods': ['ar4'],
   'factor': 0.425,
   'factor_calculation_method': 'ar4',
   'factor_calculati

# Estimates
Climatiq allows you to perform estimates based on an emission factor. You specify the emission factor using an activity id, and optionally filtering on region, year or other things.
The following code snippet will make an estimation of how much CO2e the power grid in Austria emits, for a certain amount of power generated. 
We will be using the `/estimate` endpoint. For a more comprehensive documentation about this endpoint, [click here](https://docs.climatiq.io/#estimate)

In [36]:
# The id for the emission factor. Found via the data explorer or the search endpoint
activity_id = "electricity-energy_source_grid_mix"
# We have many regions with the same activity id, representing the power grid in different countries.
# We'd like to get the power for Australia specifically, so we will need to specify a region. We do this by specifying the UN location code for the region
# You can also see the region for different emission factors in the data explorer
region = "AT"

# We must also specify how much electricity generation we would like to make the estimation for.
# In this case we will do it for 100kilowattt-hours. 
# Different emission factors have different requirement as to what units they support estimates for
# You can see the units supported by an emission factor in the data explorer, and find more documentation about units
# in the API documentation.
parameters = {
        "energy": 100,
        "energy_unit": "kWh"
    }

json_body = {
    "emission_factor": {
        "id": activity_id,
        "region": region,
        # More filters are possible here. See the full documentation for more options
    },
    # Specify how much energy we're estimating for
    "parameters": parameters
}

# You must always specify your AUTH token in the "Authorization" header like this.
authorization_headers = {"Authorization": f"Bearer: {MY_API_KEY}"}

# We send a POST request to the estimate endpoint with a json body and the correct authorization headers
response = requests.post("https://beta3.api.climatiq.io/estimate", json=json_body, headers=authorization_headers)
# The response is also json.
pprint.pprint(response.json())



{'co2e': 4.8688,
 'co2e_calculation_method': 'ar4',
 'co2e_calculation_origin': 'source',
 'co2e_unit': 'kg',
 'constituent_gases': {'ch4': None,
                       'co2': None,
                       'co2e_other': None,
                       'co2e_total': 4.8688,
                       'n2o': None},
 'emission_factor': {'category': 'Electricity',
                     'id': 'electricity-energy_source_grid_mix',
                     'lca_activity': 'well_to_tank',
                     'region': 'AT',
                     'source': 'BEIS',
                     'year': '2021'}}


# Estimate response
As you can see above, the estimate response is a JSON response that consists of three parts:
- The outermost part, which is the emissions for your estimation
- The `constituent_gases` which is additional information the source provides (if any) about what other greenhouse gases are emitted. You shouldn't worry too much about this field unless you know you need this.
- The `emission_factor`, which is Climatiq specifying what emission factor was used. You can see that the `id` and the `region` matches what we specified in our API call.


# Batch estimate
The Climatiq API is very fast, but if you have a lot of data it might be problematic to make an API call for every estimation. Climatiq has a batch endpoint where you can make up to 100 estimations per API call.
It will return a result for each estimation in the same order that the estimates were sent.
[More details about the batch estimate endpoint here](https://docs.climatiq.io/#batch-estimations)

Below we will make three estimations with the same emission factor, but different energy amounts

In [37]:
activity_id = "electricity-energy_source_grid_mix"
region = "AT"

parameters = {
        "energy": 100,
        "energy_unit": "kWh"
    }

# Notice that the JSON body here is an array, with each entry being an estimate
json_body = [
{
    "emission_factor": {
        "id": activity_id,
        "region": region,
    },
    "parameters": {
        "energy": 10,
        "energy_unit": "kWh"
    }
},
{
    "emission_factor": {
        "id": activity_id,
        "region": region,
    },
    "parameters": {
        "energy": 100,
        "energy_unit": "kWh"
    }
}
]

# You must always specify your AUTH token in the "Authorization" header like this.
authorization_headers = {"Authorization": f"Bearer: {MY_API_KEY}"}

# We send a POST request to the /batch endpoint with a json body and the correct authorization headers
response = requests.post("https://beta3.api.climatiq.io/batch", json=json_body, headers=authorization_headers)
# The response is also json.
response.json()



{'results': [{'co2e': 0.48688000000000003,
   'co2e_unit': 'kg',
   'co2e_calculation_method': 'ar4',
   'co2e_calculation_origin': 'source',
   'emission_factor': {'id': 'electricity-energy_source_grid_mix',
    'source': 'BEIS',
    'year': '2021',
    'region': 'AT',
    'category': 'Electricity',
    'lca_activity': 'well_to_tank'},
   'constituent_gases': {'co2e_total': 0.48688000000000003,
    'co2e_other': None,
    'co2': None,
    'ch4': None,
    'n2o': None}},
  {'co2e': 4.8688,
   'co2e_unit': 'kg',
   'co2e_calculation_method': 'ar4',
   'co2e_calculation_origin': 'source',
   'emission_factor': {'id': 'electricity-energy_source_grid_mix',
    'source': 'BEIS',
    'year': '2021',
    'region': 'AT',
    'category': 'Electricity',
    'lca_activity': 'well_to_tank'},
   'constituent_gases': {'co2e_total': 4.8688,
    'co2e_other': None,
    'co2': None,
    'ch4': None,
    'n2o': None}}]}

# Moving on
Climatiq has much more to offer, such as:
- Over 3500 emission factors covering purchsaes, fuel, accomodation, different travel modes, waste processing [and more](https://docs.climatiq.io/#examples).
- Estimating based on [flight routes](https://docs.climatiq.io/#flights)
- Estimating the emissions of [cloud compute usage](https://docs.climatiq.io/#cloud-computing)
- Estimating by using industry classification codes such as ISIC or NAICS